# GeoPandas

Por: [Eduardo Graells-Garrido](http://datagramas.cl) (`egraells@udd.cl`).
  
Data Sources:

  * [Biblioteca del Congreso Nacional](https://www.bcn.cl/siit/mapas_vectoriales/index_html)
  * [Biblioteca SECTRA](http://www.sectra.gob.cl/encuestas_movilidad/encuestas_movilidad.htm)
  * [GTFS Santiago](http://datos.gob.cl/dataset/33245)


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Esto configura la apariencia de los gráficos utilizando configuraciones de seaborn
sns.set(context='poster', style='ticks', palette='inferno', font='Linux Biolinum O')

# Esto es una instrucción de Jupyter que hace que los gráficos se desplieguen en el notebook
%matplotlib inline

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 150

## Proyecciones y Sistemas de Referencia

  * Chile: <https://epsg.io/5361>
  * Latitud/Longitud, nivel mundial: <https://epsg.io/4326>

In [ ]:
!ls -l ./input/urban_areas/

In [ ]:
urban_areas = gpd.read_file('./input/urban_areas/', crs={'init': 'epsg:5361'})
urban_areas

In [ ]:
urban_areas.plot()

In [ ]:
urban_areas_ll = urban_areas.to_crs({'init': 'epsg:4326'})

In [ ]:
urban_areas_ll.plot()

In [ ]:
urban_areas_ll[urban_areas.NOMBRE == 'Santiago'].plot()

## Operaciones Geográficas

In [ ]:
municipalities = gpd.read_file('./input/shapefile_comunas/', crs={'init': 'epsg:5361'})
municipalities.plot()

In [ ]:
municipalities[municipalities.NOM_PROV.isin(['Santiago', 'Maipo', 'Cordillera'])].plot()
plt.axis('equal');

In [ ]:
stgo_municipalities = gpd.sjoin(municipalities, urban_areas[urban_areas.NOMBRE == 'Santiago'], 
                                op='intersects')
stgo_municipalities.shape

In [ ]:
stgo_municipalities.plot(edgecolor='white')

In [ ]:
ax = stgo_municipalities.plot(figsize=(12,12), edgecolor='black', facecolor='white')
urban_areas[urban_areas.NOMBRE == 'Santiago'].plot(ax=ax, alpha=0.5, color='grey')

In [ ]:
stgo = urban_areas[urban_areas.NOMBRE == 'Santiago'].geometry.values[0]
stgo

In [ ]:
def intersection(municipality, urban_area):
    return municipality.intersection(urban_area)

stgo_municipalities.head().geometry.map(lambda x: intersection(x, stgo))

In [ ]:
def urban_part(municipality):
    return intersection(municipality, stgo)

In [ ]:
stgo_urban_municipalities = (stgo_municipalities.copy()
                             .assign(geometry=lambda df_: df_['geometry'].map(urban_part))
                             .pipe(lambda x: x[~x.NOM_COM.isin(['Lampa', 'Colina'])])
                             .to_crs({'init': 'epsg:4326'}))
stgo_urban_municipalities.shape

In [ ]:
stgo_urban_municipalities.plot(column='NOM_PROV', edgecolor='white')
#plt.axis('equal')

In [ ]:
stgo_urban_municipalities.sample(5)

## Rutas

In [ ]:
# credits to Diego for this

from shapely.geometry import LineString
import geopandas as gpd
import pandas as pd

# @todo: add day of the week (calendar.txt)
# @done: filter by route intersected with a polygon
#        this can be done using intersection afterwards
# @done: return the name of each route (nombre recorrido ej: C02c)


def gtfs_routes(gtfspath='input/gtfsv34'):
    # agencia que entrega la informacion
    agency = pd.read_csv(gtfspath + '/agency.txt')
    
    # forma del recorrido (geometria)
    shapes = pd.read_csv(gtfspath + '/shapes.txt')
    
    # nombre del recorrido (metadatos)
    routes = pd.read_csv(gtfspath + '/routes.txt')
    
    # metadatos viajes de cada recorrido, existen para cada ruta
    trips = pd.read_csv(gtfspath + '/trips.txt')

    trips_routes = trips.drop_duplicates('shape_id').merge(routes)
    # there are many trips with the same shape because a route can be
    # repeated during the day, we only keep one of those.
    # Keep in mind that this will need to be improved for retrieving the
    # day of the week.
    shapes_trips_routes = shapes.merge(trips_routes)
    
    toLineString = lambda x: LineString(x.loc[:,('shape_pt_lon', 'shape_pt_lat')].values)

    # classic lat/lon
    crs_latlon = {'init': 'epsg:4326'}
    lines = gpd.GeoSeries(shapes_trips_routes.groupby('shape_id').apply(toLineString), crs=crs_latlon)
    route_data = (shapes_trips_routes.drop_duplicates('shape_id').set_index('shape_id')
                               .loc[:,('route_color','route_short_name','agency_id')] )
    route_data['route_color'] = '#' + route_data['route_color']
    shapes_agency_gdf = gpd.GeoDataFrame(route_data, geometry = lines).reset_index()
    return shapes_agency_gdf

In [ ]:
gtfs = gtfs_routes()

In [ ]:
gtfs.head()

In [ ]:
gtfs.plot(linewidth=1)

In [ ]:
ax = None
for idx, group in gtfs.groupby('route_color'):
    ax = group.plot(color=idx, ax=ax, linewidth=1)

In [ ]:
from carpynter import plot_lines

In [ ]:
plot_lines(gtfs, color_column='route_color', figsize=14)

In [ ]:
def plot_routes(gtfs, ax, color=None, linewidth=None):
    for idx, group in gtfs.groupby('route_color'):
        group.plot(color=idx if color is None else color, ax=ax, linewidth=linewidth)  

ax = stgo_urban_municipalities.plot(figsize=(18,16), color='#EFEFEF', edgecolor='#444444')
plot_routes(gtfs[gtfs.agency_id == 'TS'], ax, linewidth=1, color='#777777')
plot_routes(gtfs[gtfs.agency_id == 'M'], ax, linewidth=4)
ax.set_axis_off()